In [1]:
from math import log2

### 计算数据集的整体信息熵

1. 计算每一个标签值的个数

2. 计算每个标签值的出现频率  pi = label[i] / count(label) 

3. 计算信息熵 Entropy = sum(-pi * log(pi))

In [2]:
def label_count_calculation(dataset):
    label_count = {}                             #计算每个标签值的个数
    for data in dataset:                   #遍历每一个数据
        value = data[-1]                   #取出每一行数据的最后一个值作为标签值
        if value not in label_count.keys():      #将标签值作为字典的键，出现次数作为字典的值，计算每一个标签值的个数
            label_count[value] = 1
        else:
            label_count[value] += 1
    return label_count

In [3]:
def entropy_calculation(dataset):
    '''
    count： 字典，key为各个不同的标签值，value为各个标签值出现的次数
    p： 每个标签值出现的频率组成的列表
    entropy：信息熵
    '''
    count = label_count_calculation(dataset)
    dataset_count = len(dataset)
    p = []

    entropy = 0.
    for val in count.values():             #遍历count中的每一个值，即每一个标签值的个数
        pi = val/dataset_count 
        entropy_pi = -pi * log2(pi)        #计算信息熵
        entropy += entropy_pi
    return entropy

### 计算信息增益

1. 计算不同特征值的个数

2. 依次取出每个特征值所包含数据的行标，并分别组成不同的小数据集

3. 计算分割后的数据集占总数据集的比例 p_data

4. 用gain_calculation循环计算每个特征下每个取值的信息增益，并返回最大信息增益

In [4]:
def data_entropy(dataset, feat, value):
    data_1 = []
    data_2 = []
    
    for index in dataset:
        if index[feat] == value:
            data_1.append(index)
        else:
            data_2.append(index)
    p_data = len(data_1)/len(dataset)
    
    result = entropy_calculation(data_1) * p_data
    return result

In [5]:
def gain_calculation(dataset):
    '''
    n：特征数
    dataset_entropy：数据集信息熵
    max_gain：最大信息增益
    max_feature：得到最大信息增益的特征索引
    feature_values：max_feature特征下的取值
    '''
    n = len(dataset[0])-1
    dataset_entropy = entropy_calculation(dataset)
    max_gain = 0.

    for feat in range(n):

        #计算每个特征下的取值
        values = list(set(example[feat] for example in dataset))
        feature_gain = dataset_entropy
        for valu in values:
            entropy = data_entropy(dataset, feat, valu)
            feature_gain -= entropy

        if feature_gain > max_gain:
            max_gain = feature_gain
            best_feature = feat
            feature_values = values

    return best_feature, feature_values

In [6]:
def gain_rate_calculation(dataset):
    '''
    n：特征数
    dataset_entropy：数据集信息熵
    max_gain_rate：最大信息增益率
    max_feature：得到最大信息增益率的特征索引
    feature_values：max_feature特征下的取值
    '''
    n = len(dataset[0])-1
    dataset_entropy = entropy_calculation(dataset)
    max_gain_rate = 0.

    for feat in range(n):

        #计算每个特征下的取值
        values = list(set(example[feat] for example in dataset))
        feature_gain = dataset_entropy
        for valu in values:
            entropy = data_entropy(dataset, feat, valu)
            feature_gain -= entropy
        feature_gain_rate = feature_gain/len(values)

        if feature_gain_rate > max_gain_rate:
            max_gain_rate = feature_gain_rate
            best_feature = feat
            feature_values = values

    return best_feature, feature_values

In [7]:
def gini_calculation(dataset):
    '''
    min_gini：最小信息增益
    best_feature：获得最小基尼指数的特征
    best_feat_valu：获得最小基尼指数的特征取值
    '''
    n = len(dataset[0]) - 1
    min_gini = 1.
    
    for feat in range(n):
        values = list(set(example[feat] for example in dataset))
        
        for valu in values:
            gini = data_entropy(dataset, feat, valu, 'CART')
            
            if gini < min_gini:
                min_gini = gini
                best_feature = feat
                best_feat_valu = valu
                
    return best_feature, best_feat_valu

In [8]:
def split_tree(dataset, axis, value):

    subtree = []          #分裂后的子树
    for feat in dataset:
        if feat[axis] == value:
            data = feat[:axis]
            data.extend(feat[axis+1:])
            subtree.append(data)
    return subtree

In [9]:
def vote(dataset):
    labelcount = label_count_calculation(dataset)
    class_predict = max(labelcount)
    return class_predict

In [10]:
def create_tree(dataset, feature_name, criterion):
    #计算出子树所包含的标签字典
    label_count = label_count_calculation(dataset)
    
    #如果子树中标签值全部相同，返回该标签值
    if len(label_count) == 1:
        return list(label_count.keys())[0]
    
    if len(dataset[0]) == 1:                                 
        # 当只剩一个特征时，返回标签值
        return max(vote(dataset))
    
    #根据不同算法获取特征索引和特征值（去重）
    if criterion == 'ID3':
        best_feature, feature_values = gain_calculation(dataset)
    elif criterion == 'C4.5':
        best_feature, feature_values = gain_rate_calculation(dataset)
    else:
        raise ValueError('criterion input error')
        
    #获取特征名字，并在原feature_name中删除该特征名字
    #记录一个坑，这里如果用del删除的话会连同feature_name一起修改
    name = feature_name[best_feature]
    new_name = feature_name[:best_feature]
    new_name.extend(feature_name[best_feature+1:])
    
    #将信息增益最大的特征索引用于构建决策树字典的键
    mytree = {name:{}}
    for valu in feature_values:
        subtree = split_tree(dataset, best_feature, valu)
        if not subtree:
            continue

        mytree[name][valu] = create_tree(subtree, new_name, criterion)
    return mytree

In [11]:
def classify(tree, data, feat_name):
    print(tree)
    if type(tree) != dict:
        return tree
    else:
        for i, key in enumerate(feat_name):
            if key in tree.keys():
                
                if len(tree[key]) == 1:
                    return list(tree.values())[0]
                else:
                    
                    new_tree = tree[key][data[i]]
                    return classify(new_tree, data, feat_name)

In [12]:
if __name__ == '__main__':
    feature_name = ['年龄','有工作','有房','信贷情况']

    dataset = [['青年','否','否','一般','否']]
    dataset.append(['青年','否','否','好','否'])
    dataset.append(['青年','是','否','好','是'])
    dataset.append(['青年','是','是','一般','是'])
    dataset.append(['青年','否','否','一般','否'])
    dataset.append(['中年','否','否','一般','否'])
    dataset.append(['中年','否','否','好','否'])
    dataset.append(['中年','是','是','好','是'])
    dataset.append(['中年','否','是','非常好','是'])
    dataset.append(['中年','否','是','非常好','是'])
    dataset.append(['老年','否','是','非常好','是'])
    dataset.append(['老年','否','是','好','是'])
    dataset.append(['老年','是','否','好','是'])
    dataset.append(['老年','是','否','非常好','是'])
    dataset.append(['老年','否','否','一般','否'])


    data = ['青年','是','否','一般']
    mytree = create_tree(dataset, feature_name, 'ID3')
    result = classify(mytree, data, feature_name)

{'有房': {'否': {'有工作': {'否': '否', '是': '是'}}, '是': '是'}}
{'有工作': {'否': '否', '是': '是'}}
是
